This script is for extracting text out of PDFs for data dictionaries



In [ ]:
import re
import pandas as pd

# Raw data (paste your content here as a multiline string)
# this example is from Hennepin County's parcel data dictionary
raw_data = """
Field Description Field Description
PID Property Identification of property DIV_PEND_IND
(DIV_PEND_I)
Division/Combination/Plat Pending Indicator
PID_TEXT Significant Property Identification Number within a
quarter/quarter PR_TYP_CD1 Subrecord 1 Property Type
DIV_STATUS_DATE (DIV_STATUS) PID Creation Date PR_TYP_NM1 Subrecord 1 Property Type Description
FEATURECODE (FEATURECOD) Feature Subtype Code HMSTD_CD1 Subrecord 1 Homestead Code
STATE_CD State Precinct Code OWNER_PCT1 Subrecord 1 Owner Percent
TORRENS_TYP (TORRENS_TY) Current Title Type CONT_IND1 Subrecord 1 Contiguous Indicator
HOUSE_NO House Number of Property LAND_MV1 Subrecord 1 Estimated Market Value - Land
FRAC_HOUSE_NO (FRAC_HOUSE) Fraction of House Number of Property BLDG_MV1 Subrecord 1 Estimated Market Value - Building
STREET_NM Street Name MACH_MV1 Subrecord 1 Estimated Market Value - Machinery
CONDO_NO Unit Number TOTAL_MV1 Subrecord 1 Estimated Market Value - Total
MAILING_MUNIC_CD (MAILING_MU) Mailing Municipal Code QUAL_IMPR1 Subrecord 1 "This Old House" Qualifying Improvement
MAILING_MUNIC_NM (MAILING__1) Mailing Municipal Name VET_EXCL1 Subrecord 1 Maximum Veteran Exclusion Amount
ZIP_CD Zip Code HMS_EXCL1 Subrecord 1 Homestead Market Value Exclusion
MULTI_ADDR_IND (MULTI_ADDR) Multiple Address Indicator NET_TC1 Subrecord 1 TAX CAPACITY
OWNER_NM Owner Name NET_TAX1 Subrecord 1 Net Tax
TAXPAYER_NM (TAXPAYER_N) Taxpayer Name & Mailing Address Line 1 PR_TYP_CD2 Subrecord 2 Property Type
TAXPAYER_NM_1 (TAXPAYER_1) Taxpayer Name & Mailing Address Line 2 PR_TYP_NM2 Subrecord 2 Property Type Description
TAXPAYER_NM_2 (TAXPAYER_2) Taxpayer Name & Mailing Address Line 3 HMSTD_CD2 Subrecord 2 Homestead Code
TAXPAYER_NM_3 (TAXPAYER_3) Taxpayer Name & Mailing Address Line 4 OWNER_PCT2 Subrecord 2 Owner Percent
MUNIC_CD Municipality Code CONT_IND2 Subrecord 2 Contiguous Indicator
MUNIC_NM Municipality Name LAND_MV2 Subrecord 2 Estimated Market Value - Land
SCHOOL_DIST_NO (SCHOOL_DIS) School District Number BLDG_MV2 Subrecord 2 Estimated Market Value - Building
WATERSHED_NO (WATERSHED_) Watershed District Number MACH_MV2 Subrecord 2 Estimated Market Value - Machinery
SEWER_DIST_NO (SEWER_DIST) Sewer District Number TOTAL_MV2 Subrecord 2 Estimated Market Value - Total
TIF_PROJECT_NO (TIF_PROJEC) Tax Increment Financing Project Number QUAL_IMPR2 Subrecord 2 "This Old House" Qualifying Improvement
PROPERTY_STATUS_CD
(PROPERTY_S)
Property Status Code (0 = Current, 3 = Non-Current, D =
In Process) VET_EXCL2 Subrecord 2 Maximum Veteran Exclusion Amount
FORFEIT_LAND_IND (FORFEIT_LA) Forfeited Land Indicator HMS_EXCL2 Subrecord 2 Homestead Market Value Exclusion
CO_OP_IND Cooperative Indicator (M = Master, U = Unit) NET_TC2 Subrecord 2 TAX CAPACITY
PRI_SEC_CODE (PRI_SEC_CO) Primary/Secondary Code NET_TAX2 Subrecord 2 Net Tax
ABBREV_ADDN_NM (ABBREV_ADD) Addition Name PR_TYP_CD3 Subrecord 3 Property Type
ADDITION_NO (ADDITION_N) Addition Number PR_TYP_NM3 Subrecord 3 Property Type Description
LOT Addition Lot HMSTD_CD3 Subrecord 3 Homestead Code
BLOCK Addition Block OWNER_PCT3 Subrecord 3 Owner Percent
METES_BNDS1 (METES_BNDS) Metes/Bounds Description Line 1 CONT_IND3 Subrecord 3 Contiguous Indicator
METES_BNDS2 (METES_BN_1) Metes/Bounds Description Line 2 LAND_MV3 Subrecord 3 Estimated Market Value - Land
METES_BNDS3 (METES_BN_2) Metes/Bounds Description Line 3 BLDG_MV3 Subrecord 3 Estimated Market Value - Building
METES_BNDS4 (METES_BN_3) Metes/Bounds Description Line 4 MACH_MV3 Subrecord 3 Estimated Market Value - Machinery
MORE_METES_BNDS_IND
(MORE_METES)
Additional Metes/Bounds Indicator
TOTAL_MV3 Subrecord 3 Estimated Market Value - Total
ABSTR_TORRENS_CD (ABSTR_TORR) Original Title Type QUAL_IMPR3 Subrecord 3 "This Old House" Qualifying Improvement
BUILD_YR Year Built VET_EXCL3 Subrecord 3 Maximum Veteran Exclusion Amount
SALE_DATE Date of Last Sale HMS_EXCL3 Subrecord 3 Homestead Market Value Exclusion
SALE_PRICE Sale Price NET_TC3 Subrecord 3 TAX CAPACITY
SALE_CODE Sale Transaction Type NET_TAX3 Subrecord 3 Net Tax
SALE_CODE_NAME (SALE_CODE_) Sale Transaction Description PR_TYP_CD4 Subrecord 4 Property Type
PARCEL_AREA (PARCEL_ARE) Area (Feet) PR_TYP_NM4 Subrecord 4 Property Type Description
MKT_VAL_TOT (MKT_VAL_TO) Total Estimated Market Value HMSTD_CD4 Subrecord 4 Homestead Code
TAXABLE_VAL_TOT (TAXABLE_VA) Total TAXABLE Market Value OWNER_PCT4 Subrecord 4 Owner Percent
NET_IMPRV_AMT (NET_IMPRV_) Total Improvement Amount CONT_IND4 Subrecord 4 Contiguous Indicator
TOT_NET_TAX (TOT_NET_TA) Total Net Tax LAND_MV4 Subrecord 4 Estimated Market Value - Land
TOT_SPEC_TAX (TOT_SPEC_T) Total Special Assessments BLDG_MV4 Subrecord 4 Estimated Market Value - Building
TAX_TOT Total Tax MACH_MV4 Subrecord 4 Estimated Market Value - Machinery
NET_TAX_PD Total Tax & Special Assessments Paid TOTAL_MV4 Subrecord 4 Estimated Market Value - Total
TOT_PENALTY_PD (TOT_PENALT) Total Penalty Paid QUAL_IMPR4 Subrecord 4 "This Old House" Qualifying Improvement
EARLIEST_DELQ_YR (EARLIEST_D) Earliest Delinquent Year VET_EXCL4 Subrecord 4 Maximum Veteran Exclusion Amount
COMP_JUDG_IND (COMP_JUDG_) Composite Judgment (CJ) Indicator HMS_EXCL4 Subrecord 4 Homestead Market Value Exclusion
MTG_CO_NBR Escrow / Other Entity Code NET_TC4 Subrecord 4 TAX CAPACITY
GR_ACRE_OPEN_SPACE_CD
(GR_ACRE_OP)
Green Acre/Open Space/Platted Land/Rural Preserve NET_TAX4
Subrecord 4 Net Tax
PETITION_REVIEW_IND (PETITION_R) Petition For Review Pending Indicator LAT Latitude (Y-coordinate) of parcel centroid (SRID 4326)
TAX_ADJ_PEND_IND (TAX_ADJ_PE) Tax Adjustment Pending Indicator LON Longitude (X-coordinate) of parcel centroid (SRID 4326)
"""

# Regex to match fields (all caps with optional underscores, numbers, or parentheses)
# Updated regex to detect field names and descriptions
field_pattern = r"([A-Z][A-Z0-9_]+(?:\s\([A-Z0-9_]+\))?)\s+(.*?)(?=(?:\s+[A-Z][A-Z0-9_]+(?:\s\([A-Z0-9_]+\))?)|\Z)"

# Match fields and descriptions
matches = re.findall(field_pattern, raw_data, re.DOTALL)

# Post-process matches to handle double-column layouts
data = []
for field, description in matches:
    # Split description on new field names detected within the text
    sub_matches = re.split(r"(?=\b[A-Z][A-Z0-9_]+(?:\s\([A-Z0-9_]+\))?\b)", description)
    # Add each detected pair as a new row
    data.append([field.strip(), sub_matches[0].strip()])
    for extra in sub_matches[1:]:
        sub_field_match = re.match(r"([A-Z][A-Z0-9_]+(?:\s\([A-Z0-9_]+\))?)\s+(.*)", extra)
        if sub_field_match:
            data.append([sub_field_match.group(1).strip(), sub_field_match.group(2).strip()])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Field", "Description"])

# Save to CSV
df.to_csv("parsed_table.csv", index=False)

# Display the DataFrame
print(df)

In [1]:
import re
import pandas as pd

# Raw data (paste your content here)
raw_data = """
Attribute:
Attribute Label: PID
Attribute Definition: Property identification of property
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PID_TEXT
Attribute Definition: Significant property identification number within a quarter/quarter section
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: DIV_STATUS_DATE
Attribute Definition: Date of when the PID was created or divided
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: FEATURECODE
Attribute Definition: Feature subtype code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: STATE_CD
Attribute Definition: State precinct code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TORRENS_TYP
Attribute Definition: Current title type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HOUSE_NO
Attribute Definition: House number of property
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: FRAC_HOUSE_NO
Attribute Definition: Fraction of house number of property
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: STREET_NM
Attribute Definition: Street name
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CONDO_NO
Attribute Definition: Condo unit number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MAILING_MUNIC_CD
Attribute Definition: Mailing address municipality code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MAILING_MUNIC_NM
Attribute Definition: Mailing address municipality name
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: ZIP_CD
Attribute Definition: Zip Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MULTI_ADDR_IND
Attribute Definition: Multiple Address Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: OWNER_NM
Attribute Definition: Property owner name
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAXPAYER_NM
Attribute Definition: Taxpayer Name and Mailing Address Line 1
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAXPAYER_NM_1
Attribute Definition: Taxpayer Name and Mailing Address Line 2
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAXPAYER_NM_2
Attribute Definition: Taxpayer Name and Mailing Address Line 3
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAXPAYER_NM_3
Attribute Definition: Taxpayer Name and Mailing Address Line 4
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MUNIC_CD
Attribute Definition: Municipality Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MUNIC_NM
Attribute Definition: Municipality Name
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SCHOOL_DIST_NO
Attribute Definition: School District Number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: WATERSHED_NO
Attribute Definition: Watershed District Number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SEWER_DIST_NO
Attribute Definition: Sewer District Number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TIF_PROJECT_NO
Attribute Definition: Tax Increment Financing Project Number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PROPERTY_STATUS_CD
Attribute Definition: Property Status Code (0 = Current, 3 = Non-Current, D = In Process)
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: FORFEIT_LAND_IND
Attribute Definition: Forfeited Land Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CO_OP_IND
Attribute Definition: Cooperative Indicator (M = Master, U = Unit)
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PRI_SEC_CODE
Attribute Definition: Primary or Secondary Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: ABBREV_ADDN_NM
Attribute Definition: Addition Name
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: ADDITION_NO
Attribute Definition: Addition Number
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LOT
Attribute Definition: Addition Lot
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BLOCK
Attribute Definition: Addition Block
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: METES_BNDS1
Attribute Definition: Metes and Bounds Description Line 1
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: METES_BNDS2
Attribute Definition: Metes and Bounds Description Line 2
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: METES_BNDS3
Attribute Definition: Metes and Bounds Description Line 3
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: METES_BNDS4
Attribute Definition: Metes and Bounds Description Line 4
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MORE_METES_BNDS_IND
Attribute Definition: Additional Metes/Bounds Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: ABSTR_TORRENS_CD
Attribute Definition: Original Title Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BUILD_YR
Attribute Definition: Year Built
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SALE_DATE
Attribute Definition: Date of Last Sale
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SALE_PRICE
Attribute Definition: Sale Price
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SALE_CODE
Attribute Definition: Sale Transaction Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: SALE_CODE_NAME
Attribute Definition: Sale Transaction Description
Attribute Definition Source: String
Attribute:
Attribute Label: PARCEL_AREA
Attribute Definition: Area in Feet
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MKT_VAL_TOT
Attribute Definition: Total ESTIMATED Market Value
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAXABLE_VAL_TOT
Attribute Definition: Total TAXABLE Market Value
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_IMPRV_AMT
Attribute Definition: Total Improvement Amount
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOT_NET_TAX
Attribute Definition: Total Net Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOT_SPEC_TAX
Attribute Definition: Total Special Assessments
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAX_TOT
Attribute Definition: Total Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TAX_PD
Attribute Definition: Total Tax and Special Assessments Paid
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOT_PENALTY_PD
Attribute Definition: Total Penalty Paid
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: EARLIEST_DELQ_YR
Attribute Definition: Earliest Delinquent Year
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: COMP_JUDG_IND
Attribute Definition: Composite Judgement (CJ) Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MTG_CO_NBR
Attribute Definition: Escrow/Other Entity Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: GR_ACRE_OPEN_SPACE_CD
Attribute Definition: Green Acre/Open Space/Platted Land/Rural Preserve
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PETITION_REVIEW_IND
Attribute Definition: Petition For Review Pending Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TAX_ADJ_PEND_IND
Attribute Definition: Tax Adjustment Pending Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: DIV_PEND_IND
Attribute Definition: Division/Combination/Plat Pending Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_CD1
Attribute Definition: Subrecord 1 Property Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_NM1
Attribute Definition: Subrecord 1 Property Type Description
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMSTD_CD1
Attribute Definition: Subrecord 1 Homestead Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: OWNER_PCT1
Attribute Definition: Subrecord 1 Owner Percent
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CONT_IND1
Attribute Definition: Subrecord 1 Contiguous Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LAND_MV1
Attribute Definition: Subrecord 1 ESTIMATED Market Value - Land
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BLDG_MV1
Attribute Definition: Subrecord 1 ESTIMATED Market Value - Building
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MACH_MV1
Attribute Definition: Subrecord 1 ESTIMATED Market Value - Machinery
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOTAL_MV1
Attribute Definition: Subrecord 1 ESTIMATED Market Value - Total
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: QUAL_IMPR1
"Attribute Definition: Subrecord 1 ""This Old House"" Qualifying Improvement"
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: VET_EXCL1
Attribute Definition: Subrecord 1 Maximum Veteran Exclusion Amount
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMS_EXCL1
Attribute Definition: Subrecord 1 Homestead Market Value Exclusion
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TC1
Attribute Definition: Subrecord 1 TAX CAPACITY
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TAX1
Attribute Definition: Subrecord 1 Net Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_CD2
Attribute Definition: Subrecord 2 Property Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_NM2
Attribute Definition: Subrecord 2 Property Type Description
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMSTD_CD2
Attribute Definition: Subrecord 2 Homestead Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: OWNER_PCT2
Attribute Definition: Subrecord 2 Owner Percent
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CONT_IND2
Attribute Definition: Subrecord 2 Contiguous Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LAND_MV2
Attribute Definition: Subrecord 2 ESTIMATED Market Value - Land
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BLDG_MV2
Attribute Definition: Subrecord 2 ESTIMATED Market Value - Building
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MACH_MV2
Attribute Definition: Subrecord 2 ESTIMATED Market Value - Machinery
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOTAL_MV2
Attribute Definition: Subrecord 2 ESTIMATED Market Value - Total
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: QUAL_IMPR2
"Attribute Definition: Subrecord 2 ""This Old House"" Qualifying Improvement"
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: VET_EXCL2
Attribute Definition: Subrecord 2 Maximum Veteran Exclusion Amount
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMS_EXCL2
Attribute Definition: Subrecord 2 Homestead Market Value Exclusion
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TC2
Attribute Definition: Subrecord 2 TAX CAPACITY
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TAX2
Attribute Definition: Subrecord 2 Net Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_CD3
Attribute Definition: Subrecord 3 Property Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_NM3
Attribute Definition: Subrecord 3 Property Type Description
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMSTD_CD3
Attribute Definition: Subrecord 3 Homestead Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: OWNER_PCT3
Attribute Definition: Subrecord 3 Owner Percent
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CONT_IND3
Attribute Definition: Subrecord 3 Contiguous Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LAND_MV3
Attribute Definition: Subrecord 3 ESTIMATED Market Value - Land
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BLDG_MV3
Attribute Definition: Subrecord 3 ESTIMATED Market Value - Building
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MACH_MV3
Attribute Definition: Subrecord 3 ESTIMATED Market Value - Machinery
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOTAL_MV3
Attribute Definition: Subrecord 3 ESTIMATED Market Value - Total
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: QUAL_IMPR3
"Attribute Definition: Subrecord 3 ""This Old House"" Qualifying Improvement"
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: VET_EXCL3
Attribute Definition: Subrecord 3 Maximum Veteran Exclusion Amount
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMS_EXCL3
Attribute Definition: Subrecord 3 Homestead Market Value Exclusion
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TC3
Attribute Definition: Subrecord 3 TAX CAPACITY
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TAX3
Attribute Definition: Subrecord 3 Net Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_CD4
Attribute Definition: Subrecord 4 Property Type
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: PR_TYP_NM4
Attribute Definition: Subrecord 4 Property Type Description
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMSTD_CD4
Attribute Definition: Subrecord 4 Homestead Code
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: OWNER_PCT4
Attribute Definition: Subrecord 4 Owner Percent
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: CONT_IND4
Attribute Definition: Subrecord 4 Contiguous Indicator
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LAND_MV4
Attribute Definition: Subrecord 4 ESTIMATED Market Value - Land
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: BLDG_MV4
Attribute Definition: Subrecord 4 ESTIMATED Market Value - Building
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: MACH_MV4
Attribute Definition: Subrecord 4 ESTIMATED Market Value - Machinery
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: TOTAL_MV4
Attribute Definition: Subrecord 3 ESTIMATED Market Value - Total
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: QUAL_IMPR4
"Attribute Definition: Subrecord 4 ""This Old House"" Qualifying Improvement"
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: VET_EXCL4
Attribute Definition: Subrecord 4 Maximum Veteran Exclusion Amount
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: HMS_EXCL4
Attribute Definition: Subrecord 4 Homestead Market Value Exclusion
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TC4
Attribute Definition: Subrecord 4 TAX CAPACITY
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: NET_TAX4
Attribute Definition: Subrecord 4 Net Tax
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: Shape
Attribute Definition: Feature Shape
Attribute Definition Source: Esri
Attribute:
Attribute Label: LAT
Attribute Definition: Latitude (Y-coordinate) of parcel centroid (SRID 4326)
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: LON
Attribute Definition: Longitude (X-coordinate) of parcel centroid (SRID 4326)
Attribute Definition Source: Hennepin County Resident Services
Attribute:
Attribute Label: Shape.STArea()
Attribute Definition: Feature shape area in meters
Attribute Definition Source: Esri
Attribute:
Attribute Label: Shape.STLength()
Attribute Definition: Feature shape perimeter length in meters
Attribute Definition Source: Esri
"""  

# Regex pattern to match the required fields
attribute_pattern = r"Attribute Label:\s*(.*?)\s*Attribute Definition:\s*(.*?)\s*Attribute Definition Source:\s*(.*?)\s*(?=Attribute:|$)"

# Find matches
matches = re.findall(attribute_pattern, raw_data, re.DOTALL)

# Create a list of dictionaries for each attribute
data = [{"Attribute Label": match[0], "Attribute Definition": match[1], "Attribute Definition Source": match[2]} for match in matches]

# Convert to a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("attributes_table.csv", index=False)

# Display the DataFrame
print(df)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
      Attribute Label                               Attribute Definition  \
0                 PID                Property identification of property   
1            PID_TEXT  Significant property identification number wit...   
2     DIV_STATUS_DATE        Date of when the PID was created or divided   
3         FEATURECODE                               Feature subtype code   
4            STATE_CD                                State precinct code   
..                ...                       